0 - INTRODUÇÃO

1 - BIBLIOTECAS DO PROJETO

Bibliotecas utilizadas para o scraping dos dados, criação e limpeza do dataframe, análise exploratória e modelagem de dados para regressão linear.

In [4]:
#Scraping
from selenium import webdriver
import time
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
#EDA
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
#Machine learning


#configurações 

pd.set_option("max_colwidth", 1000)
pd.set_option("display.max_columns", 60)
pd.set_option("display.max_rows", 20)
sns.set_style("whitegrid")
sns.despine()




<Figure size 640x480 with 0 Axes>

2 - WEB SCRAPING COM SELENIUM

In [2]:
#Criando a estrutura de scraping
service = Service()
options = webdriver.FirefoxOptions()
driver = webdriver.Firefox(options=options, service=service) #criando o webdriver
driver.get('https://www.vivareal.com.br/venda/?pagina=1') #abrindo o driver





In [3]:
#lista para armazenar os dados obtidos
imoveis_lista  = []

In [ ]:
#acessando os xpaths e coletando os dados
for p in range(1,278): #277 páginas
      imoveis = driver.find_elements(By.CLASS_NAME, "property-card__content-link")
      for imovel in imoveis:           
           titulo = imovel.find_element(By.XPATH, './/span[@class="property-card__title js-cardLink js-card-title"]').text
           price = imovel.find_element(By.XPATH, './/div[@class="property-card__price js-property-card-prices js-property-card__price-small"]').text
           area = imovel.find_element(By.XPATH, './/span[@class="property-card__detail-value js-property-card-value property-card__detail-area js-property-card-detail-area"]').text
           quartos = imovel.find_element(By.XPATH, './/li[@class="property-card__detail-item property-card__detail-room js-property-detail-rooms"]').text
           banheiros = imovel.find_element(By.XPATH, './/li[@class="property-card__detail-item property-card__detail-bathroom js-property-detail-bathroom"]').text
           garagem = imovel.find_element(By.XPATH, './/li[@class="property-card__detail-item property-card__detail-garage js-property-detail-garages"]').text
           endereco = imovel.find_element(By.XPATH, './/span[@class="property-card__address"]').text
          
           imovel_view = {
                 'titulo':titulo,
                 "preco":price,
                 "area": area,
                 'quartos':quartos,
                 'banheiros': banheiros,
                 'garagem': garagem,
                 'endereco': endereco     
            }
           imoveis_lista.append(imovel_view)
      driver.find_element(By.XPATH, '/html/body/main/div[2]/div[1]/section/div[2]/div[2]/div/ul/li[9]/button').click()
      time.sleep(5)
     




3 - CRIANDO DATAFRAME

In [ ]:
imoveis_br = pd.DataFrame(imoveis_lista)
imoveis_br

In [22]:
#verificando dados duplicados
imoveis_br.duplicated().value_counts()

False    8946
True     1056
Name: count, dtype: int64

In [24]:
#removendo dados duplicados
imoveis_br = imoveis_br.drop_duplicates()
imoveis_br

,titulo,preco,area,quartos,banheiros,garagem,endereco
0,"Apartamento com 3 Quartos à Venda, 150m²",R$ 320.000\nPreço abaixo do mercado,150,3 Quartos,4 Banheiros,4 Vagas,"Travessa Felício, 41 - Cascadura, Rio de Janei..."
1,"Casa com 3 Quartos à Venda, 74m²",R$ 98.556,74,3 Quartos,2 Banheiros,3 Vagas,"Rua José Bolonha, 547 - Igacaba, Pedregulho - SP"
2,"Casa com 3 Quartos à Venda, 450m²",R$ 550.000,450,3 Quartos,5 Banheiros,10 Vagas,"Avenida Ponte Alta Norte, 08 - Ponte Alta Nort..."
3,"Casa com 3 Quartos à Venda, 110m²",R$ 230.000,110,3 Quartos,2 Banheiros,1 Vaga,"Ponte Marcos Freire II, 56 - Marcos Freire II,..."
4,"Apartamento com 3 Quartos à Venda, 95m²",R$ 485.000,95,3 Quartos,3 Banheiros,2 Vagas,"Avenida Presidente Kennedy, 432 - Piedade, Jab..."
...,...,...,...,...,...,...,...
9996,"Apartamento com 2 Quartos à Venda, 46m²",R$ 265.000,46,2 Quartos,1 Banheiro,1 Vaga,"Rua Cravinhos, 0 - Jardim Paulistano, Ribeirão..."
9997,"Apartamento com Quarto à Venda, 56m²",R$ 640.000,56,1 Quarto,2 Banheiros,1 Vaga,"Avenida Wladimir Meirelles Ferreira, 0 - Jardi..."
9998,"Casa de condomínio com 3 Quartos à Venda, 130m²",R$ 650.000,130,3 Quartos,1 Banheiro,2 Vagas,"Bonfim Paulista, Ribeirão Preto - SP"
9999,"Apartamento com 3 Quartos à Venda, 118m²",R$ 600.000,118,3 Quartos,3 Banheiros,1 Vaga,"Rua Humaitá, 0 - Santa Cruz do José Jacques, R..."


In [27]:
#salvando dados como csv
imoveis_br.to_csv("imoveis_br.csv")

3.1 - LIMPEZA DO DATAFRAME

In [79]:
df = pd.read_csv('imoveis_br.csv', index_col=0 )
df = df.drop([99, 1181, 3248, 3255, 5117, 5130, 5285, 5380, 5511, 5754, 7611, 7770,
       7943, 8562, 8945, 9631]) #removendo info de terrenos/lotes
df = df.reset_index(drop=True)


#verificando
df

,titulo,preco,area,quartos,banheiros,garagem,endereco
0,"Apartamento com 3 Quartos à Venda, 150m²",R$ 320.000\nPreço abaixo do mercado,150,3 Quartos,4 Banheiros,4 Vagas,"Travessa Felício, 41 - Cascadura, Rio de Janeiro - RJ"
1,"Casa com 3 Quartos à Venda, 74m²",R$ 98.556,74,3 Quartos,2 Banheiros,3 Vagas,"Rua José Bolonha, 547 - Igacaba, Pedregulho - SP"
2,"Casa com 3 Quartos à Venda, 450m²",R$ 550.000,450,3 Quartos,5 Banheiros,10 Vagas,"Avenida Ponte Alta Norte, 08 - Ponte Alta Norte, Brasília - DF"
3,"Casa com 3 Quartos à Venda, 110m²",R$ 230.000,110,3 Quartos,2 Banheiros,1 Vaga,"Ponte Marcos Freire II, 56 - Marcos Freire II, Nossa Senhora do Socorro - SE"
4,"Apartamento com 3 Quartos à Venda, 95m²",R$ 485.000,95,3 Quartos,3 Banheiros,2 Vagas,"Avenida Presidente Kennedy, 432 - Piedade, Jaboatão dos Guararapes - PE"
...,...,...,...,...,...,...,...
8925,"Apartamento com 2 Quartos à Venda, 46m²",R$ 265.000,46,2 Quartos,1 Banheiro,1 Vaga,"Rua Cravinhos, 0 - Jardim Paulistano, Ribeirão Preto - SP"
8926,"Apartamento com Quarto à Venda, 56m²",R$ 640.000,56,1 Quarto,2 Banheiros,1 Vaga,"Avenida Wladimir Meirelles Ferreira, 0 - Jardim Botânico, Ribeirão Preto - SP"
8927,"Casa de condomínio com 3 Quartos à Venda, 130m²",R$ 650.000,130,3 Quartos,1 Banheiro,2 Vagas,"Bonfim Paulista, Ribeirão Preto - SP"
8928,"Apartamento com 3 Quartos à Venda, 118m²",R$ 600.000,118,3 Quartos,3 Banheiros,1 Vaga,"Rua Humaitá, 0 - Santa Cruz do José Jacques, Ribeirão Preto - SP"


In [234]:
df.dtypes

titulo        object
preco        float64
area           int32
quartos        int32
banheiros      int32
garagem        int32
endereco      object
dtype: object

3.2 - LIMPEZA DA FEATURE 'TITULO'

In [80]:
df['titulo']

0              Apartamento com 3 Quartos à Venda, 150m²
1                      Casa com 3 Quartos à Venda, 74m²
2                     Casa com 3 Quartos à Venda, 450m²
3                     Casa com 3 Quartos à Venda, 110m²
4               Apartamento com 3 Quartos à Venda, 95m²
                             ...                       
8925            Apartamento com 2 Quartos à Venda, 46m²
8926               Apartamento com Quarto à Venda, 56m²
8927    Casa de condomínio com 3 Quartos à Venda, 130m²
8928           Apartamento com 3 Quartos à Venda, 118m²
8929    Casa de condomínio com 3 Quartos à Venda, 272m²
Name: titulo, Length: 8930, dtype: object

In [81]:
#tipo de dado armazenado
type(df['titulo'][0])

str

É esperado e desejado que o título seja, de fato, uma string. Quando a isso a coluna está adequada. Farei apenas uma mudança estética na coluna para melhor leitura.

In [82]:
#substituindo "," por " com"
titulo = []
for t in df['titulo']:
    t = t.replace(",", " com")
    titulo.append(t)

In [235]:
df['titulo'] = titulo
df['titulo'] = df['titulo'].astype(str)

3.3 - LIMPEZA DA FEATURE 'PRECO'

In [84]:
df["preco"]

0       R$ 320.000\nPreço abaixo do mercado
1                                 R$ 98.556
2                                R$ 550.000
3                                R$ 230.000
4                                R$ 485.000
                       ...                 
8925                             R$ 265.000
8926                             R$ 640.000
8927                             R$ 650.000
8928                             R$ 600.000
8929                           R$ 1.500.000
Name: preco, Length: 8930, dtype: object

In [85]:
#tipo armazenado
type(df["preco"][0])

str

É esperado que a feature 'preco' seja do tipo numérico (float) para isso é necessário limpar os 'R$' e 'Preço abaixo do mercado'.

In [86]:
#removendo cifrão
preco = [p.replace("R$ ","") for p in df["preco"]]
preco = [p.replace("\nPreço abaixo do mercado", "") for p in preco]


df["preco"] = preco

In [87]:
#Verificando
df["preco"]


0         320.000
1          98.556
2         550.000
3         230.000
4         485.000
          ...    
8925      265.000
8926      640.000
8927      650.000
8928      600.000
8929    1.500.000
Name: preco, Length: 8930, dtype: object

In [88]:
type(df["preco"][0])

str

A aparência dos dados contidos em 'preco' está correta, porém o tipo de dado não é o adequado ainda.

In [89]:
df["preco"]

0         320.000
1          98.556
2         550.000
3         230.000
4         485.000
          ...    
8925      265.000
8926      640.000
8927      650.000
8928      600.000
8929    1.500.000
Name: preco, Length: 8930, dtype: object

In [90]:
#Necessário retirar o ponto para comunicar a dimensão correta dos dados
df["preco"] = [p.replace(".","") for p in df["preco"]]
df["preco"] = df["preco"].astype(float)

In [91]:
df["preco"]

0        320000.0
1         98556.0
2        550000.0
3        230000.0
4        485000.0
          ...    
8925     265000.0
8926     640000.0
8927     650000.0
8928     600000.0
8929    1500000.0
Name: preco, Length: 8930, dtype: float64

Dados agora estão no formato correto.

3.4 - LIMPEZA DA FEATURE 'area'

In [92]:
df['area']

0       150
1        74
2       450
3       110
4        95
       ... 
8925     46
8926     56
8927    130
8928    118
8929    272
Name: area, Length: 8930, dtype: object

In [93]:
#verificando o tipo de dado
type(df['area'][0])

str

Apesar de termos número inteiros na coluna, o tipo de dado delas está como str. 

In [94]:
#fazendo a mudança do tipo de dados e salvando
df['area'] = df['area'].astype(int)

In [95]:
#verificando
type(df['area'][0])

numpy.int32

3.5 - LIMPEZA DA  FEATURE 'quartos'

In [97]:
df['quartos']

0       3 Quartos
1       3 Quartos
2       3 Quartos
3       3 Quartos
4       3 Quartos
          ...    
8925    2 Quartos
8926     1 Quarto
8927    3 Quartos
8928    3 Quartos
8929    3 Quartos
Name: quartos, Length: 8930, dtype: object

É preciso remover a string 'Quartos' e converter para o tipo int.

In [99]:
#removendo str
df['quartos'] = [q.replace(" Quartos","") for q in df['quartos']]
df['quartos'] = [q.replace(" Quarto", "") for q in df['quartos']]
df['quartos'] = [q.replace("--", "0") for q in df['quartos']]

Todas as ocorrências de 'quartos' == '--' são de terrenos/loteamentos indicando que não existem quartos nesses imoveis pois ainda não foram construídos.

In [107]:
#mudando o tipo de dados
df['quartos'] = df['quartos'].astype(int)
df['quartos'].dtypes

dtype('int32')

3.6 - LIMPEZA DA FEATURE 'BANHEIROS'

In [108]:
df['banheiros']

0       4 Banheiros
1       2 Banheiros
2       5 Banheiros
3       2 Banheiros
4       3 Banheiros
           ...     
8925     1 Banheiro
8926    2 Banheiros
8927     1 Banheiro
8928    3 Banheiros
8929    4 Banheiros
Name: banheiros, Length: 8930, dtype: object

In [109]:
#removendo as str
df['banheiros'] = [q.replace(" Banheiros","") for q in df['banheiros']]
df['banheiros'] = [q.replace(" Banheiro", "") for q in df['banheiros']]
df['banheiros'] = [q.replace("--", "0") for q in df['banheiros']]

Assim como na feature anterior temos '--' indicando loteamentos e terrenos. O tratamento seguiu o mesmo.

In [111]:
#Mudando o tipo de dado
df['banheiros'] = df['banheiros'].astype(int)
df['banheiros'].dtypes

dtype('int32')

3.7 - LIMPEZA DA FEATURE 'GARAGEM'

In [115]:
df['garagem']

0        4 Vagas
1        3 Vagas
2       10 Vagas
3         1 Vaga
4        2 Vagas
          ...   
8925      1 Vaga
8926      1 Vaga
8927     2 Vagas
8928      1 Vaga
8929     5 Vagas
Name: garagem, Length: 8930, dtype: object

In [116]:
df['garagem'] = [q.replace(" Vagas","") for q in df['garagem']]
df['garagem'] = [q.replace(" Vaga", "") for q in df['garagem']]
df['garagem'] = [q.replace("--", "0") for q in df['garagem']]

In [232]:
df['garagem'] = df['garagem'].astype(int)
df['garagem'].dtypes

dtype('int32')

In [119]:
df.columns

Index(['titulo', 'preco', 'area', 'quartos', 'banheiros', 'garagem',
       'endereco'],
      dtype='object')

3.8 - LIMPEZA DA FEATURE 'ENDERECO'

Essa feature será dissolvida e outras três novas adicionadas, são elas: Rua, Cidade e Estado.

In [205]:
df['endereco']

0                               Travessa Felício, 41 - Cascadura, Rio de Janeiro - RJ
1                                    Rua José Bolonha, 547 - Igacaba, Pedregulho - SP
2                      Avenida Ponte Alta Norte, 08 - Ponte Alta Norte, Brasília - DF
3        Ponte Marcos Freire II, 56 - Marcos Freire II, Nossa Senhora do Socorro - SE
4             Avenida Presidente Kennedy, 432 - Piedade, Jaboatão dos Guararapes - PE
                                            ...                                      
8925                        Rua Cravinhos, 0 - Jardim Paulistano, Ribeirão Preto - SP
8926    Avenida Wladimir Meirelles Ferreira, 0 - Jardim Botânico, Ribeirão Preto - SP
8927                                             Bonfim Paulista, Ribeirão Preto - SP
8928                 Rua Humaitá, 0 - Santa Cruz do José Jacques, Ribeirão Preto - SP
8929                Rua Arlindo Rivoiro, 0 - Recreio das Acácias, Ribeirão Preto - SP
Name: endereco, Length: 8930, dtype: object

In [216]:
ender_split = df['endereco'].str.split("-")
for e in ender_split:
    if len(e) == 1:
        print(e)

['BR 116,']
['SP']
['Zona Rural, SP']


In [212]:
uf = []
for i in ender_split:
    uf.append(i[-1].strip())

cidade_bairro = []
for i in ender_split:
    if len(i) == 3:
        cidade_bairro.append(i[-2].strip())
    if len(i) == 2:
        cidade_bairro.append(i[0].strip())   

bairro = []
cidade = []
for i in cidade_bairro:
    lista_bairro_cidade = i.split(sep=",")
    bairro.append(lista_bairro_cidade[0])
    cidade.append(lista_bairro_cidade[1]) 

IndexError: list index out of range

In [213]:
lista_bairro_cidade

['Sarandi']